# Week 7 worksheet: SciPy and pandas for data science and statistics

This week, we introduce the modules **SciPy** and **pandas**, which provide convenient tools for statistics, optimisation, and data science.

The best way to learn programming is to write code. Don't hesitate to edit the code in the example cells, or add your own code, to test your understanding. You will find practice exercises throughout the notebook, denoted by 🚩 ***Exercise $x$:***.

#### Displaying solutions

Solutions will be released one week after the worksheets are released, as a new `.txt` file in the same GitHub repository. After pulling the file to your workspace (either your computer or your Noteable server), run the following cell to create clickable buttons under each exercise, which will allow you to reveal the solutions.

In [ ]:
%run scripts/create_widgets.py week07

---
## SciPy

The **SciPy** library provides a set of scientific computing tools for Python -- [it is actually a relative of Numpy](https://www.scipy.org/scipylib/faq.html#numpy-vs-scipy-vs-other-packages). I would recommend you have a browse of the documentation and tutorial linked below to get an idea of all the functionalities included. For this course, we will give a brief tour of some of SciPy's modules in action, to give you a sense of how they might be used to solve practical problems.

---
**📚 Learn more:**
* [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/)
* [SciPy tutorial - Introduction](https://docs.scipy.org/doc/scipy/reference/tutorial/general.html)
* [Frequently asked questions - SciPy documentation](https://www.scipy.org/scipylib/faq.html#frequently-asked-questions)
* [Optimization](https://docs.scipy.org/doc/scipy/reference/optimize.html) - This contains functions for finding minima (or maxima) and roots of equations in one or more dimensions.
* [Interpolation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) - Contains functions for interpolating values using splines or other approximating polynomials.
* [Statistics](https://docs.scipy.org/doc/scipy/reference/stats.html) - Many more statistics related functions.
* [Integration](https://docs.scipy.org/doc/scipy/reference/integrate.html) - Contains functions for performing integration (using Simpson's rule and more complicated quadrature rules).

---

### Interpolating some noisy data

Suppose we have a set of data points $(x_i, y_i)$, with coordinates stored in vectors `x` and `y`. The **`interpolate` sub-module** provides tools to interpolate data. In particular, it contains many functions for constructing piecewise polynomials, and in particular [splines](https://en.wikipedia.org/wiki/Spline_(mathematics)).

Below is an example of how spline interpolation might be used:

In [ ]:
# First, import all necessary modules
import numpy as np
import scipy.interpolate
import scipy as scp
%matplotlib notebook
import matplotlib.pyplot as plt

# Generate some data that looks like a smooth curve with some 
# noise overlayed:
def generate_data(x):
    '''
    Generate some smooth data (sin curve) with overlaid noise 
    coming from a Normal distribution.
    '''
    # Generate some noise from a Normal distribution - 
    # this generates an array of values simulated from 
    # an N(0, 0.3**2), with the array of the same length as x:
    noise = np.random.normal(0, 0.2, len(x))
    y = np.sin(x) + noise
    return y

# Set up x and generate y data:
step = 0.4
x = np.arange(0, 10 + step, step)
y = generate_data(x)

# Make an interpolating function - this one goes through all of the 
# data points, and joins them with a piecewise cubic function:
f_interp = scp.interpolate.interp1d(x, y, kind='cubic')

# Now fit a spline using a different function that takes a smoothing 
# parameter s, this spline won't go through all of the points but will 
# look smoother:
f_smooth = scp.interpolate.UnivariateSpline(x, y, s=4)

# Now suppose we want a value for y at a point in between
# our x values - let's say x = 1.5:
print(f_interp(1.5))
print(f_smooth(1.5))

# Make a denser x-axis to plot our interpolated functions
xmin, xmax = x[0], x[-1]
dense_step = 0.05
x_dense = np.arange(xmin, xmax + dense_step, dense_step)

# Plot the data
fig, ax = plt.subplots()

ax.plot(x, y, 'k.', label='Noisy data')
ax.plot(x_dense, np.sin(x_dense), 'g-', label='Underlying function')
ax.plot(x_dense, f_interp(x_dense), 'c--', label='Interpolating spline')
ax.plot(x_dense, f_smooth(x_dense), 'r-.', label='Smooth spline')

ax.set_xlabel(r'$x$')
ax.set_ylabel(r'$y$')
ax.set_xlim([xmin, xmax])
ax.legend(loc='upper center')

plt.show()

Note that the interpolating functions (here, `scp.interpolate.interp1d()` and `scp.interpolate.UnivariateSpline`) **return a function object**, which we can then call as any other function.

Make sure to look up the two spline fitting functions to check you understand how the call here matches what is in the documentation.

---
***🚩 Exercise:***
* Try a different function as the smooth background function from `np.sin(x)`.
* Modify the function `generate_data()` to also take the standard deviation of the noise as input. Experiment with different values, and observe how well or poorly the splines fit the data, depending on how noisy it is.
* Try changing the smoothing parameter `s` in the second fitted spline. How does this change the result? Can you make it look like the first interpolating spline?

---

### Summary statistics, reading a file

Next, let's look at some descriptive statistics (means, variance, etc). To do that, we can examine some real data -- the file `oil_reserve_data.csv` should have been downloaded to your Noteable workspace (and should therefore be in the same directory as this notebook) when you fetched this week's assignment. The file is a slightly edited version of one that comes from [this EU data source](http://data.europa.eu/euodp/en/data/dataset/RH8xH4aVauYKO92JJxv8sA). It describes the emergency oil reserves kept by several European countries over recent years.

In [ ]:
import numpy as np
import scipy.stats as stats

# Use Numpy's loadtxt function to open the data file:
oil_data = np.loadtxt('oil_reserve_data.csv',
                      dtype=float,
                      delimiter=',',
                      skiprows=1,
                      usecols=(1, 2, 3, 4))

Note that we have to skip a row and not use the first column in order to get a section of this file that is all of floating point type. Have a look at the file e.g. in Excel to see what it looks like. (In the next section, we will see better ways to import this type of data, using the `pandas` module.)

In [ ]:
# Unpack each column into a different array named for the 
# country depicted in the data (use transpose to obtain the
# array as a list of rows):
germany, denmark, belgium, bulgaria = oil_data.T

# Use the describe function to get basic summary statistics:
print('stats.describe output:')
print('Germany:\n', stats.describe(germany))
print('Denmark:\n', stats.describe(denmark))

# Find the 5% and 95% percentile locations:
print('\n5% and 95% percentiles:')
print(np.percentile(germany, (5, 95)))

# Find the Pearson's and Spearman's correlation coefficients
# between two columns:
print('\nCorrelation coefficients for Germany/Denmark and Germany/Bulgaria:')
print(stats.pearsonr(germany, denmark))
print(stats.spearmanr(germany, bulgaria))

# We can save out an array to a file using Numpy's
# savetxt() function:
np.savetxt('belgium.txt', belgium)

---
🚩 ***Exercise 1:***
- What is the median value for Bulgaria? the mode? the geometric mean?
- Try finding correlations between the different columns.
- Can you find a linear relationship between the data from Germany and Denmark (i.e., can you fit a linear function $y = ax + b$, where $x$ is the data for Germany and $y$ the data for Bulgaria)?

Look for the relevant functions in the documentation for Numpy and for `scipy.stats`.

In [ ]:
%run scripts/show_solutions.py week07_ex1

---
### Optimisation

The module `scipy.optimize` may be of interest for those of you interested in optimisation, for instance in operational research. For example, let's try one of the problems from [this page](http://people.brunel.ac.uk/~mastjjb/jeb/or/morelp.html):

> A carpenter makes tables and chairs. Each table can be sold for a profit of £30 and each chair for a profit of £10.
> 1. The carpenter can afford to spend up to 40 hours per week working and takes six hours to make a table and three hours to make a chair.
> 2. Customer demand requires that they make at least three times as many chairs as tables.
> 3. Tables take up four times as much storage space as chairs and there is room for at most four tables each week.

The goal is to **maximise** the profit that the carpenter can make in 1 week, by finding the optimal number of tables $x_T$ and number of chairs $x_C$ that they should make.

This is a *linear programming problem*: we need to maximise the weekly profit, given by the objective function $f(x_T, x_C) = 30x_T + 10x_C$, subject to 3 linear constraints:

$$
\begin{align}
0. & & x_T &\geq 0, x_C \geq 0 \\
1. & & 6x_T + 3x_C &\leq 40 \\
2. & & x_C &\geq 3x_T \\
3. & & x_C/4 + x_T &\leq 4
\end{align}
$$

We can use [`scipy.optimize.linprog()`](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html#linear-programming-linprog) to solve this problem, but first we need to rearrange it a little bit, to fit the kind of input that this function takes.

$$
\begin{align}
&\min_{x_T, x_C} -30x_T - 10x_C, \\
&\text{such that} \quad
\begin{bmatrix}
-1 & 0 \\
0 & -1 \\
6 & 3 \\
3 & -1 \\
1 & \frac{1}{4}
\end{bmatrix}
\begin{bmatrix}
x_T \\ x_C
\end{bmatrix}
\leq
\begin{bmatrix}
0 \\ 0 \\ 40 \\ 0 \\ 4
\end{bmatrix}.
\end{align}
$$

Looking at the documentation, the default bounds on the variable values is that they are non-negative, which corresponds to our problem here -- so we don't need to specify them.

In [ ]:
from scipy.optimize import linprog

A = np.array([[6, 3],
              [3, -1],
              [1, 0.25]])

b = np.array([40, 0, 4], dtype=float)
c = np.array([-30, -10], dtype=float)

solution = linprog(c, A, b)
print('Output of linprog:')
print(solution, '\n')

results = A @ solution.x

print(f'The optimal strategy is to make {solution.x[0]:.2f} tables',
      f'and {solution.x[1]:.2f} chairs every week (on average), for',
      f'a profit of £{-solution.fun:.2f} per week.',
      f'This is {results[0]:.1f} hours of work per week, '
      f'taking up {100*results[2]/4:.1f}% of storage space.')

There are many other optimisation and root-finding methods available in `scipy.optimise`, each more or less well-suited to different problems. When faced with such a problem, the best thing to do is to have a browse in the documentation, and find if there is perhaps a method already implemented there that you could use.

---
🚩 ***Exercise 2:*** The carpenter is aiming to make a profit of at least £150 per week, but now they have twice the storage space. What is the minimum number of hours they need to work in a week to achieve this?

What if they have to make exactly 3 chairs for each table?

In [ ]:
%run scripts/show_solutions.py week07_ex2

---
## Pandas

The second module we will look at today (and next week!) is **Pandas**. Pandas is a module which allows the construction of a **dataframe**, an object to store data that looks a little like a spreadsheet (the data is indexed principally by a column name and row name/number). The data contained in a dataframe does *not* have to be of the same type. 

---
**📚 Learn more:**
* [Pandas (main website)](http://pandas.pydata.org/index.html)
* [Pandas documentation](http://pandas.pydata.org/pandas-docs/stable/).
* [A quick introduction to Pandas](http://pandas.pydata.org/pandas-docs/stable/10min.html)
* There is a fantastic tutorial (also in Jupyter) [here](http://pandas.pydata.org/pandas-docs/stable/tutorials.html) (under Lessons for New Pandas Users). This is well worth working through a little if you want a longer introduction to the basic concepts in Pandas.
---

Here are some basic examples, the first uses the same file `oil_reserve_data.csv` from earlier.

In [ ]:
# First, import the pandas module
import pandas as pd

# Use the read_csv method to read the CSV file into a dataframe
oil_data = pd.read_csv('oil_reserve_data.csv')

# Look at what the dataframe contains
oil_data

In [ ]:
# Get the column headers
print('Column headers:')
print(oil_data.columns, '\n')

# Pull the data from a particular column by referring to it by name
print('Data from Germany:')
print(oil_data['Germany'])

We can also specify a column to use as row labels when reading the file -- note the difference with the previous command here:

In [ ]:
# Use the first column in the file as row index
oil_data = pd.read_csv('oil_reserve_data.csv', index_col=0)

# Print the column names and the row names
print(oil_data.columns)
print(oil_data.index)

# Look at what the dataframe contains
oil_data

There are plenty of other optional arguments of `pd.read_csv()` which are very helpful to read CSV files with different properties and layouts -- [have a look at the documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).

---
### Indexing dataframes

[The user guide in the pandas documentation is a must-read](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html). Here is a summary:

- **`.iloc`** is used for indexing by number (like in Numpy arrays).

In [ ]:
oil_data = pd.read_csv('oil_reserve_data.csv', index_col=0)

# Select a row from our data, by numerical index (here, the second row)
print(oil_data.iloc[1], '\n')

# We can also grab a column by number - here, the second column
print(oil_data.iloc[:, 1])

- **`.loc`** is used for indexing by label (row or column header), and also for Boolean indexing.

In the second example here:
- we use `.loc` to return all the data for Bulgaria,
- we create a Boolean dataframe using `<` which is `True` where the reserves for Bulgaria are below 1000,
- we use `.index` with the Boolean array to return the names of all the **rows** (i.e. the dates) where this happened.

In [ ]:
# Print data for June 2019 in Germany, using row and column labels
print(oil_data.loc['2019M06', 'Germany'], '\n')

# Print the dates when the reserves in Bulgaria are below 1000
bulgaria_below_1000 = oil_data.loc[:, 'Bulgaria'] < 1000
print(bulgaria_below_1000, '\n')
print(oil_data.index[bulgaria_below_1000], '\n')

Note that `.columns`, `.index`, `.loc` and `.iloc` **are not functions** (i.e. you don't use parentheses when using them, but square brackets).

---
🚩 ***Exercise 3:*** Display the oil reserves in Denmark, but only at the dates when the reserves in Germany are below 20000 **and** the reserves in Belgium are above 4000. [You might find this helpful...](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing)

In [ ]:
%run scripts/show_solutions.py week07_ex3

---
Let us look at a different dataset. The file `r_and_d_spend.csv` comes from an [open European dataset](http://data.europa.eu/euodp/en/data/dataset/Lnlc8Fcv5u1RYlfjnsKxg) describing the GDP spend of different countries on research and development work.

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt

# Read in our data
randd = pd.read_csv('r_and_d_spend.csv')
print(randd)

# Plot the data for Austria
fig, ax = plt.subplots()
ax.plot(randd['Year'], randd['Austria'], 'bo')

# Fit a line through the Austria data
LR_aus = st.linregress(randd['Year'], randd['Austria'])
# slope, intercept = LR_aus[0], LR_aus[1]
y_aus = LR_aus.intercept + LR_aus.slope * randd['Year']

ax.plot(randd['Year'], y_aus, 'b-')
plt.show()

---
**📚 Learn more:**
* [pandas.DataFrame.loc - Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html)
* [pandas.DataFrame.iloc - Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html)
* [Indexing, iteration - Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html#indexing-iteration)
* [Plotting - Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html#plotting)
* [Boolean indexing - Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing)

---
🚩 ***Exercise 4:*** Look at the other columns in the `r_and_d_spend.csv` file. Fit linear best fit lines to the other columns, make plots of them similar to the one shown above. Try putting them all in the same figure but on different subplots. Add legends, titles, and axis labels as necessary.

In [ ]:
%run scripts/show_solutions.py week07_ex4

---
🚩 ***Exercise 5:*** Plot the data for all countries on the same graph between 2000 and 2010.

In [ ]:
%run scripts/show_solutions.py week07_ex5